In [12]:
import pandas as pd
import numpy as np
import os
import shutil
from glob import glob
import bentoml

In [11]:
!pip install bentoml

  Using cached bentoml-1.0.5-py3-none-any.whl (792 kB)
  Using cached opentelemetry_api-1.12.0-py3-none-any.whl (52 kB)
  Using cached prometheus_client-0.13.1-py3-none-any.whl (57 kB)
  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)
  Using cached watchfiles-0.16.1-cp37-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.2 MB)
  Using cached opentelemetry_sdk-1.12.0-py3-none-any.whl (81 kB)
  Using cached uvicorn-0.18.3-py3-none-any.whl (57 kB)
  Using cached python_multipart-0.0.5-py3-none-any.whl
  Using cached opentelemetry_instrumentation-0.33b0-py3-none-any.whl (23 kB)
  Using cached opentelemetry_util_http-0.33b0-py3-none-any.whl (6.6 kB)
  Using cached pip_tools-6.8.0-py3-none-any.whl (51 kB)
  Using cached deepmerge-1.0.1-py3-none-any.whl (8.0 kB)
  Using cached opentelemetry_instrumentation_asgi-0.33b0-py3-none-any.whl (8.9 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached simple_di-0.1.5-py3-none-any.whl (9.8 kB)
  Using cached starlette

    Uninstalling prometheus-client-0.14.1:
      Successfully uninstalled prometheus-client-0.14.1
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.


In [3]:
os.listdir('./Data')

['.c',
 'boats',
 'buoy',
 'cruise ship',
 'Data_preprocessing.ipynb',
 'ferry boat',
 'freight boat',
 'gondola',
 'inflatable boat',
 'kayak',
 'paper boat',
 'sailboat']

In [7]:
categories = glob('./Data/*')

In [8]:
categories

['./Data/boats',
 './Data/buoy',
 './Data/cruise ship',
 './Data/Data_preprocessing.ipynb',
 './Data/ferry boat',
 './Data/freight boat',
 './Data/gondola',
 './Data/inflatable boat',
 './Data/kayak',
 './Data/paper boat',
 './Data/sailboat']

In [9]:
len(categories)

11

In [13]:
filedf = pd.DataFrame()
for cat in categories:
    files = glob(cat+"/*")
    tempdf = pd.DataFrame({'filepath':files,'category':cat.split("/")[-1]})
    filedf = pd.concat([filedf,tempdf])

In [14]:
filedf.head()

,filepath,category
0,./Data/boats/buoy,boats
1,./Data/boats/cruise ship,boats
2,./Data/boats/ferry boat,boats
3,./Data/boats/freight boat,boats
4,./Data/boats/gondola,boats


In [15]:
filedf['category'].value_counts()

sailboat           488
kayak              254
gondola            242
cruise ship        239
ferry boat          81
buoy                68
paper boat          40
freight boat        29
inflatable boat     21
boats                9
Name: category, dtype: int64

In [16]:
gby_cnt = filedf.groupby("category").aggregate('count').rename(columns = {'filepath':'cnt'}).reset_index().sort_values(by='cnt',ascending=False)

In [17]:
gby_cnt 

,category,cnt
9,sailboat,488
7,kayak,254
5,gondola,242
2,cruise ship,239
3,ferry boat,81
1,buoy,68
8,paper boat,40
4,freight boat,29
6,inflatable boat,21
0,boats,9


* renmove categories having imgaes < 40

In [18]:
focus_cats = gby_cnt[gby_cnt['cnt'] >=40]['category'].values
filedf  = filedf[filedf['category'].isin(focus_cats)]
filedf['category'].value_counts()

sailboat       488
kayak          254
gondola        242
cruise ship    239
ferry boat      81
buoy            68
paper boat      40
Name: category, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, _, _ = train_test_split(filedf, filedf['category'],stratify=filedf['category'], test_size=0.4)

X_test, X_val, _, _ = train_test_split(X_test, X_test['category'], stratify=X_test['category'], test_size=0.5)

In [20]:
X_train['type'] = 'train'
X_val['type'] = 'val'
X_test['type'] = 'test'

In [21]:
fulldf = pd.concat([X_train,X_test,X_val])

In [22]:
fulldf.head()

,filepath,category,type
108,./Data/cruise ship/cruise-ship-travel-sea-vaca...,cruise ship,train
13,./Data/paper boat/origami-boat-fold-folding-pa...,paper boat,train
18,./Data/kayak/boat-kayak-inuit-transportation-k...,kayak,train
25,./Data/sailboat/boat-ocean-recreation-sailboat...,sailboat,train
199,./Data/gondola/venice-italy-gondola-outdoor-16...,gondola,train


## Creating train val test folders

In [23]:
!mkdir image
!rm -rf image/
!rm -rf image/train
!rm -rf image/test
!rm -rf image/val
!mkdir image/
!mkdir image/train
!mkdir image/test
!mkdir image/val

In [24]:
fulldf.category.unique()

array(['cruise ship', 'paper boat', 'kayak', 'sailboat', 'gondola',
       'buoy', 'ferry boat'], dtype=object)

In [25]:
from tqdm import tqdm
for cat in tqdm(fulldf.category.unique()):
    os.system(f"mkdir image/train/'{cat}'") 
    os.system(f"mkdir image/test/'{cat}'") 
    os.system(f"mkdir image/val/'{cat}'") 

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 112.16it/s]


## copy images to respective train test and validation folders

In [26]:
for i,row in tqdm(fulldf.iterrows()):
    # Boat category
    cat = row['category']
    # section is train,val or test
    section = row['type']
    # input filepath to copy
    ipath = row['filepath']
    # output filepath to paste
    opath = ipath.replace(f"Data/",f"image/{section}/")
    # running the cp command
    os.system(f"cp '{ipath}' '{opath}'")